In [ ]:
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img=cv2.imread('/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/rohit_sharma/5e5443ebaeeb1b3ea42415d013fc3437.jpg')
img.shape

(1024, 710, 3)

In [ ]:
plt.imshow(img)

In [ ]:
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
plt.imshow(gray,cmap='gray')

import haarascascade from opencv

In [ ]:
face_cascade= cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/colab dataset/model/opencv/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade= cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/colab dataset/model/opencv/haarcascades/haarcascade_eye.xml')
faces=face_cascade.detectMultiScale(gray,1.3,5)
faces

array([[153, 181, 365, 365]], dtype=int32)

In [ ]:
(x,y,w,h)=faces[0]
face_img=cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
plt.imshow(face_img)

In [ ]:
for (x,y,w,h) in faces:
  img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  roi_gray = gray[y:y+h, x:x+w]
  roi_color = img[y:y+h, x:x+w]
  eyes = eye_cascade.detectMultiScale(roi_gray)
  for (ex,ey,ew,eh) in eyes:
    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

plt.figure()
plt.imshow(face_img, cmap='gray')
plt.show()



In [ ]:
%matplotlib inline
plt.imshow(roi_color, cmap='gray') #crop the face region by roi_color(region of interest) that is red

Write a function that takes input a normal image and return cropped image

In [ ]:
import os
def get_cropped_image_if_2_eyes(image_path):
  if not os.path.exists(image_path):
    print(f"Error: Image file does not exist at {image_path}")
    return None
  img = cv2.imread(image_path)
  if img is None:
    print(f"Error: Unable to read image file at {image_path}")
    return None

  img=cv2.imread(image_path)
  gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces=face_cascade.detectMultiScale(gray,1.3,5)
  for(x,y,w,h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    if len(eyes)>=2:      #if there are 2 or more eyes then only consider that image
       return roi_color

In [ ]:
cropped_image=get_cropped_image_if_2_eyes('/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/virat_kohli/Virat_Kohli_portrait.jpg')
plt.imshow(cropped_image)

make a cropped folder to store all the images


In [ ]:
path_to_data="/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/"
path_to_cr_data="/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/cropped/"

In [ ]:
#to get complete path of individual player folder
import os
img_dirs=[]
for entry in os.scandir(path_to_data):
  if entry.is_dir():
    img_dirs.append(entry.path)

In [ ]:
img_dirs

['/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/maahi',
 '/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/lionel_messi',
 '/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/virat_kohli',
 '/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/rohit_sharma',
 '/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/cristiano_ronaldo',
 '/content/drive/MyDrive/Colab Notebooks/colab dataset/model/datasets/cropped']

In [ ]:
#if folder exists then replace it with cropped folder
import shutil
if not os.path.exists(path_to_cr_data):
  os.mkdir(path_to_cr_data)

In [ ]:
#iterate through image directories
cropped_image_dirs=[]
celebrity_file_names_dict={}
for img_dir in img_dirs:
  count=1
  celebrity_name=img_dir.split('/')[-1]  #split the address into two tokens and use last token that will be name of celibrity
  print(celebrity_name)

  celebrity_file_names_dict[celebrity_name]=[]  #dictionary
  for entry in os.scandir(img_dir):  #scandir will iterate on all the pictures
    roi_color=get_cropped_image_if_2_eyes(entry.path)
    if roi_color is not None:
      cropped_folder=path_to_cr_data+celebrity_name  #create cropped folder for particular celebrity
      if not os.path.exists(cropped_folder):
        os.makedirs(cropped_folder)
        cropped_image_dirs.append(cropped_folder)
        print("Generating cropped images in folder: ",cropped_folder)
      cropped_file_name=celebrity_name+str(count)+".png"      #store the cropped file as messi1.png messi2.png
      cropped_file_path=cropped_folder+"/"+cropped_file_name
      #used for first time only

      cv2.imwrite(cropped_file_path, roi_color)  #store imamges
      celebrity_file_names_dict[celebrity_name].append(cropped_file_path)  #make a dictionary having key as name and value as array of path of all the images stored
      count+=1

maahi
lionel_messi
virat_kohli
rohit_sharma
cristiano_ronaldo


Now the images are cropped but it also contains images of other people faces and also blurred images

so we have to remove those images manually


In [ ]:
#Now to feature extraction of cleaned images by wavelet transformation(one of the most effective feature engineering
#this func input an image to perform wavelet transformation then it will return a new image that is wavelet transformed
#wavelt uses fourier transformation with some modification

import numpy as np
import pywt
import cv2

def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)
    imArray /= 255;
    # compute coefficients
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)
    coeffs_H[0] *= 0;

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H
im_har = w2d(cropped_image,'db1',5)
plt.imshow(im_har, cmap='gray')



#this help to find structure of face

We stack raw image vertically on wavelet image by using dictionary in cropped folder

In [ ]:
celebrity_file_names_dict

In [ ]:
#store celebrity name as dictionary
class_dict={}
count=0
for celebrity_name in celebrity_file_names_dict.keys():
  class_dict[celebrity_name]=count
  count=count+1
class_dict

In [ ]:
#now iterate through the dictionary
X=[]
y=[]

for celebrity_name,training_files in celebrity_file_names_dict.items(): #iterate through perosn
  for training_image in training_files:  #iterate through every image of that person
    img=cv2.imread(training_image)  #read and scale raw images to same size
    if(img) is None:
      continue
    scalled_raw_img=cv2.resize(img,(32,32))
    img_har=w2d(img,'db1',5)  #wavelet transformation image
    scalled_img_har=cv2.resize(img_har,(32,32))
    combined_img=np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
    X.append(combined_img)
    y.append(class_dict[celebrity_name])




In [ ]:
len(X)

In [ ]:
X=np.array(X).reshape(len(X),4096).astype(float)  #convert to float to avoid warning by sklearn
X.shape

Now Train the model

first we will use support vector machine because it works good with classifications

then use grid search cv to tune parameter and use other models

In [ ]:

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)
#use pipeline to scale
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel = 'rbf', C = 10))])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.8103448275862069

In [ ]:
print(classification_report(y_test, pipe.predict(X_test)))
#this shows accuracy, precison,recall for different celebrity

              precision    recall  f1-score   support

           0       0.62      1.00      0.77        10
           1       1.00      0.75      0.86        16
           2       0.88      0.78      0.82         9
           3       0.82      0.75      0.78        12
           4       0.82      0.82      0.82        11

    accuracy                           0.81        58
   macro avg       0.83      0.82      0.81        58
weighted avg       0.84      0.81      0.81        58



In [ ]:

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
#grid search cv
model_params={
    'svm':{
        'model':svm.SVC(gamma='auto',probability=True),
        'params': {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'randomforestclassifier__n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'logisticregression__C': [1,5,10]
        }
    }
}

In [ ]:
scores = []
best_estimators = {}
import pandas as pd
for algo, mp in model_params.items(): #iterate over every model
    pipe = make_pipeline(StandardScaler(), mp['model']) #first create pipeline to sclae data
    clf =  GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False) #use 5-folds
    clf.fit(X_train, y_train)
    scores.append({
        'model': algo,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[algo] = clf.best_estimator_

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.747227,"{'svc__C': 1, 'svc__kernel': 'linear'}"
1,random_forest,0.580672,{'randomforestclassifier__n_estimators': 10}
2,logistic_regression,0.805042,{'logisticregression__C': 1}


In [ ]:
best_estimators['logistic_regression'].score(X_test,y_test)

0.8275862068965517

In [ ]:
best_model=best_estimators['logistic_regression']

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,best_model.predict(X_test))
cm

array([[ 7,  1,  0,  2,  0],
       [ 0, 14,  0,  1,  1],
       [ 0,  0,  8,  0,  1],
       [ 1,  1,  1,  9,  0],
       [ 1,  0,  0,  0, 10]])

In [ ]:
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(cm,annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
class_dict

{'maahi': 0,
 'lionel_messi': 1,
 'virat_kohli': 2,
 'rohit_sharma': 3,
 'cristiano_ronaldo': 4,
 'cropped': 5}

Save the model

In [ ]:
import joblib
joblib.dump(best_model,'saved_model.pkl')

['saved_model.pkl']

save class dictionary

In [ ]:
import json
with open("class_dictionary.json","w") as f:
  f.write(json.dumps(class_dict))